# 🎨 EchoCanvas Data Preparation & Enrichment

Welcome to the data preparation notebook for the EchoCanvas art dataset! This notebook guides you step-by-step through the process of transforming raw art data into a rich, research-ready dataset. Each section is carefully documented to help you understand the rationale, methods, and outcomes of every data engineering and enrichment step.

Whether you're an art enthusiast, data scientist, or educator, you'll find clear explanations and practical code for:
- Filtering and structuring the dataset
- Enriching metadata from WikiArt and Wikipedia
- Handling missing values and biographies
- Ensuring the dataset is ready for analysis or creative projects

Let's get started!

## Step 1: Load and Explore the Raw Dataset

We begin by loading the full dataset and identifying the most complete sample. This helps us understand the structure and quality of the data before any cleaning or filtering.


In [4]:
import pandas as pd

# Filepath to the CSV file
file_path = "/Users/chrisloukasntais/VsCode/EchoCanvas/full_dataset.csv"

# Load the CSV file
df = pd.read_csv(file_path)

# Count the number of non-null values in each row
df["non_null_count"] = df.notnull().sum(axis=1)

# Find the row with the maximum number of non-null values
row_with_most_values = df.loc[df["non_null_count"].idxmax()]

# Drop the helper column before displaying
row_with_most_values = row_with_most_values.drop("non_null_count")

# Display the row
print("First sample with the most columns having values:")
print(row_with_most_values)

/var/folders/bc/8_g2sm9x4917kl9d15xv74cc0000gn/T/ipykernel_1348/3387213093.py:7: DtypeWarning: Columns (3,5,6,7,10,12,13,15,17,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


First sample with the most columns having values:
author_name                                                Pablo Picasso
painting_name                    Blind Minotaur Is Guided By Girl 1934 2
image_url              https://uploads5.wikiart.org/images/pablo-pica...
Genre                                              mythological painting
Style                                                         Surrealism
Nationality                                                      Spanish
PaintingSchool                                            École de Paris
ArtMovement                                                       Cubism
Field                                         painting,sculpture,drawing
Date                                                                1934
Influencedby           Marc Chagall,Henri Rousseau,El Greco,Francisco...
Media                                                      etching,paper
Influencedon           Marc Chagall,Amedeo Modigliani,Willem de Kooni...
F

# Step 2: Create the Initial Filtered Dataset

Here, we filter the raw data to include only paintings with essential information (author, title, style, date, and image). We also select artists with a moderate number of works, ensuring a balanced and diverse dataset for further enrichment.

In [2]:
import pandas as pd

# Filepath to the CSV file
file_path = "/Users/chrisloukasntais/VsCode/EchoCanvas/data/full_dataset.csv"

# Load the CSV file
df = pd.read_csv(file_path, low_memory=False)

# Define required and desired output columns
required_columns = ["author_name", "painting_name", "Style","Date","image_url"]
output_columns = ["author_name", "painting_name", "Style", "Genre", "Theme", "Date", "Period", "image_url"]

# Drop rows with missing values in required columns
df_filtered = df.dropna(subset=required_columns)

# Count paintings per author
painter_counts = df_filtered["author_name"].value_counts()

# Filter painters with 4 to 10 paintings
eligible_painters = painter_counts[(painter_counts >= 4) & (painter_counts <= 6)].index

# Select rows for eligible painters
eligible_df = df_filtered[df_filtered["author_name"].isin(eligible_painters)]

# Take up to 10 paintings per painter
final_df = eligible_df.groupby("author_name").head(10).loc[:, output_columns]

# Save the result
output_file_path = "/Users/chrisloukasntais/VsCode/EchoCanvas/data/dataset.csv"
final_df.to_csv(output_file_path, index=False)

print(f"Filtered dataset saved to: {output_file_path}")


Filtered dataset saved to: /Users/chrisloukasntais/VsCode/EchoCanvas/data/dataset.csv


### Convert csv file to json form

## Step 3: Convert CSV to JSON Format

To facilitate downstream processing and integration with web applications, we convert the filtered CSV dataset into a structured JSON file.

In [11]:
import pandas as pd

# Load the CSV file (replace with your actual file path)
csv_file = "/Users/chrisloukasntais/VsCode/EchoCanvas/data/dataset.csv"
df = pd.read_csv(csv_file)

# Save as JSON 
json_file = "/Users/chrisloukasntais/VsCode/EchoCanvas/data/dataset.json"
df.to_json(json_file, orient="records", indent=4)

print(f"CSV data saved as JSON in '{json_file}'")


CSV data saved as JSON in '/Users/chrisloukasntais/VsCode/EchoCanvas/data/dataset.json'


In [12]:
import json

try:
    with open('/Users/chrisloukasntais/VsCode/EchoCanvas/data/dataset.json', 'r') as file:
        data = json.load(file)
    print("JSON is valid!")
except json.JSONDecodeError as e:
    print(f"JSON is invalid: {e}")

JSON is valid!


### Ger info about the json file

## Step 4: Analyze the JSON Dataset

We inspect the JSON dataset for missing values and identify which artists or paintings require further attention. This step ensures data completeness and highlights areas for enrichment.

In [1]:
import pandas as pd
import json

# Filepath to the JSON file
json_file_path = "/Users/chrisloukasntais/VsCode/EchoCanvas/data/dataset.json"

# Load the JSON file into a DataFrame
with open(json_file_path, 'r') as file:
    data = json.load(file)

df = pd.DataFrame(data)

# Check for missing values in each column
missing_values = df.isnull().sum()
print("Missing values per feature:")
print(missing_values)

# Get the names of painters with missing values in any column
painters_with_missing_values = df[df.isnull().any(axis=1)]["author_name"].unique()
print("\nPainters with missing values:")
print(painters_with_missing_values)

# Group by painter and list their paintings with missing values
print("\nPaintings with missing values for each painter:")
for painter in painters_with_missing_values:
    painter_paintings = df[(df["author_name"] == painter) & (df.isnull().any(axis=1))]
    print(f"\nPainter: {painter}")
    print(painter_paintings[["painting_name", "Theme", "Date", "image_url"]])  # Adjust

Missing values per feature:
author_name        0
painting_name      0
Style              0
Genre             39
Theme            655
Date              43
Period           645
image_url          0
dtype: int64

Painters with missing values:
['Fanny Sanín' 'Juan Sánchez Cotán' 'Pedro de Mena' 'Wolfgang Lettl'
 'Kirsten Justesen' 'Cennino Cennini' 'Giulia Lama' 'Filippo Cifariello'
 'Yeghishe Tadevosyan' 'Constantin Artachino' 'Thalia Flora Karavia'
 'Matej Sternen' 'Nikolai Kulbin' 'Margaret Macdonald' 'Sydney Laurence'
 'Panos Terlemezian' 'Benjamin Brown' 'Louis Vivin' 'Louis Wain'
 'Rubens Santoro' 'Mildred Anne Butler' 'Julian Falat'
 'Eduard Von Grutzner' 'Robert Bateman' 'Alexandre Jacques Chantron'
 'Anton Mauve' 'Charles Hermans' 'Franz Richard Unterberger'
 'Henry Moore Ra' 'Konstantin Dmitriyevich Flavitsky'
 'August Friedrich Schenck' 'John Varley' 'Mikhail Shibanov'
 'Christoffer Wilhelm Eckersberg' 'Constantin Daniel Rosenthal'
 'Gustave Boulanger' 'Manuel Rodriguez De Guzma

In [12]:
import pandas as pd

# Load the dataset
file_path = "/Users/chrisloukasntais/VsCode/EchoCanvas/data/dataset.csv"
df = pd.read_csv(file_path)

# Count unique artist names
unique_artists = df['author_name'].nunique()

print(f"Number of different artist names: {unique_artists}")

Number of different artist names: 132


## Step 5: Enrich Metadata Using WikiArt

To enhance our dataset, we:
- Access the (unofficial) WikiArt API
- Search for each artwork by artist and title
- Retrieve additional metadata (style, genre, period, etc.)
- Collect artist biographies and other details
- Update our dataset with this enriched information

This process adds valuable context and depth to each artwork and artist.


### 📦 Install Required Python Packages

Before proceeding, ensure you have the following packages installed:

```bash
pip install requests pandas tqdm wikipedia
```

These libraries are essential for data processing and API access.


## Step 7: Accessing the WikiArt API

WikiArt does not offer an official public API, but an open (unofficial) endpoint is available for programmatic access:

[WikiArt API Example](https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=NAME&page=1&json=2)

We'll use this to fetch metadata and biographies.


## Step 8: Prepare Artist Names for API Requests

To ensure successful API calls, we format artist names to match WikiArt's URL structure (lowercase, hyphens instead of spaces).

In [1]:
import pandas as pd

df = pd.read_csv('/Users/chrisloukasntais/VsCode/EchoCanvas/data/dataset.csv')

def format_artist_name(name):
    return name.lower().replace(" ", "-")


## Step 9: Fetch and Enrich Painting Metadata

We define functions to retrieve paintings and their metadata for each artist, then match and enrich our dataset with this new information.

In [6]:
import pandas as pd
import requests
from tqdm import tqdm
import re

def clean_title(text):
    # Lowercase, remove punctuation and extra whitespace
    return re.sub(r'[^\w\s]', '', text.lower()).strip()


def format_artist_name(name):
    return name.lower().replace(" ", "-")

def fetch_paintings(artist_url):
    paintings = []
    page = 1
    while True:
        url = f"https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl={artist_url}&page={page}&json=2"
        print(f"Fetching URL: {url}")  # Debugging: Print the URL being fetched
        response = requests.get(url)
        
        # Check if the response is successful
        if response.status_code != 200:
            print(f"Failed to fetch data. HTTP Status Code: {response.status_code}")
            print(f"Response Text: {response.text}")  # Print the response text for debugging
            break
        
        # Check if the response contains valid JSON
        try:
            data = response.json()
        except ValueError as e:
            print(f"Failed to parse JSON. Error: {e}")
            print(f"Response Text: {response.text}")  # Print the response text for debugging
            break
        
        # If no data is returned, stop fetching
        if not data:
            print("No more data returned from the API.")
            break
        
        paintings.extend(data)
        print(f"Page {page}: Retrieved {len(data)} paintings.")  # Debugging: Print the number of paintings retrieved
        page += 1
    
    print(f"Total paintings fetched for {artist_url}: {len(paintings)}")  # Debugging: Total paintings fetched
    return paintings

def match_and_enrich(row, all_paintings):
    title = clean_title(row['painting_name'].split("##")[0])
    for p in all_paintings:
        painting_title = clean_title(p['title'])
        if title == painting_title:
            row['Style'] = row['Style'] or p.get('style', '')
            row['Genre'] = row['Genre'] or p.get('genre', '')
            row['Theme'] = row['Theme'] or (', '.join(p.get('subjects', [])) if p.get('subjects') else '')
            row['Period'] = row['Period'] or p.get('period', '')
            return row
    return row

def enrich_dataset(csv_path, output_path):
    df = pd.read_csv(csv_path)
    enriched_rows = []
    artists = df['author_name'].unique()

    artist_paintings = {}

    print("\nFetching metadata from WikiArt...")
    for artist in tqdm(artists):
        artist_url = format_artist_name(artist)
        paintings = fetch_paintings(artist_url)
        artist_paintings[artist] = paintings

    print("\nMatching and enriching metadata...")
    for _, row in tqdm(df.iterrows(), total=len(df)):
        paintings = artist_paintings.get(row['author_name'], [])
        enriched_row = match_and_enrich(row, paintings)
        enriched_rows.append(enriched_row)

    enriched_df = pd.DataFrame(enriched_rows)
    enriched_df.to_csv(output_path, index=False)
    print(f"\n✅ Metadata enrichment complete. Saved to {output_path}")


In [8]:
import ssl
print(ssl.OPENSSL_VERSION)

OpenSSL 3.5.0 8 Apr 2025


In [9]:
# Test the API connection for a specific artist
artist_name = "August Friedrich Schenck"
artist_url = format_artist_name(artist_name)
print(f"Formatted Artist URL: {artist_url}")

# Fetch paintings for the artist
paintings = fetch_paintings(artist_url)

# Check the result
if paintings:
    print(f"Successfully fetched {len(paintings)} paintings for {artist_name}.")
else:
    print(f"No paintings found for {artist_name}.")

Formatted Artist URL: august-friedrich-schenck
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=august-friedrich-schenck&page=1&json=2
Page 1: Retrieved 17 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=august-friedrich-schenck&page=2&json=2
Page 2: Retrieved 17 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=august-friedrich-schenck&page=3&json=2
Page 3: Retrieved 17 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=august-friedrich-schenck&page=4&json=2
Page 4: Retrieved 17 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=august-friedrich-schenck&page=5&json=2
Page 5: Retrieved 17 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=august-friedrich-schenck&page=6&json=2
Page 6: Retrieved 17 paintings.
Fetching URL: https://www.wikiart.o

KeyboardInterrupt: 

## Step 10: Check Artist Availability on WikiArt

We verify which artists in our dataset are present in the WikiArt database and whether their paintings have available metadata. This helps us focus enrichment efforts where data is accessible.

In [10]:
import pandas as pd
import requests
from tqdm import tqdm

def format_artist_name(name):
    """Format the artist name for the WikiArt API."""
    return name.lower().replace(" ", "-")

def check_artist_availability(artist_name):
    """Check if an artist is available in the WikiArt database."""
    artist_url = format_artist_name(artist_name)
    url = f"https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl={artist_url}&page=1&json=2"
    response = requests.get(url)
    
    # If the response is successful and contains data, the artist is available
    if response.status_code == 200 and response.json():
        return True
    return False

def find_available_artists(file_path):
    """Check availability of all artists in the dataset."""
    # Load the dataset
    df = pd.read_csv(file_path)
    
    # Get unique artist names
    artist_names = df['author_name'].unique()
    
    # Initialize lists to store results
    available_artists = []
    unavailable_artists = []
    
    print("\nChecking artist availability on WikiArt...")
    for artist in tqdm(artist_names):
        if check_artist_availability(artist):
            available_artists.append(artist)
        else:
            unavailable_artists.append(artist)
    
    # Print results
    print("\n✅ Available Artists:")
    for artist in available_artists:
        print(f"- {artist}")
    
    print("\n❌ Unavailable Artists:")
    for artist in unavailable_artists:
        print(f"- {artist}")
    
    return available_artists, unavailable_artists

# Example usage
file_path = "/Users/chrisloukasntais/VsCode/EchoCanvas/data/dataset.csv"
available_artists, unavailable_artists = find_available_artists(file_path)


Checking artist availability on WikiArt...


100%|██████████| 132/132 [01:56<00:00,  1.13it/s]


✅ Available Artists:
- Cennino Cennini
- Giulia Lama
- Filippo Cifariello
- Yeghishe Tadevosyan
- Constantin Artachino
- Thalia Flora Karavia
- Matej Sternen
- Nikolai Kulbin
- Margaret Macdonald
- Sydney Laurence
- Panos Terlemezian
- Benjamin Brown
- Louis Vivin
- Louis Wain
- Rubens Santoro
- Mildred Anne Butler
- Julian Falat
- Eduard Von Grutzner
- Robert Bateman
- Alexandre Jacques Chantron
- Anton Mauve
- Charles Hermans
- Franz Richard Unterberger
- Henry Moore Ra
- Konstantin Dmitriyevich Flavitsky
- August Friedrich Schenck
- John Varley
- Mikhail Shibanov
- Christoffer Wilhelm Eckersberg
- Constantin Daniel Rosenthal
- Gustave Boulanger
- Manuel Rodriguez De Guzman
- Frederick William Burton
- Leonardo Alenza
- Charles Martin Powell
- Pieter Van Hanselaere
- Vieira Portuense
- Marie Guillemine Benoist
- Ike No Taiga
- Jose Luzan
- David Morier
- Yosa Buson
- Juriaen Van Streeck
- Bada Shanren
- Theodoor Van Thulden
- Johannes Moreelse
- Adam Van Noort
- Osias Beert
- Barbar

## Step 11: Match Paintings Between Dataset and WikiArt

For each available artist, we fetch all paintings from WikiArt and compare them to our dataset. This step ensures that our records align with external sources and highlights any missing or unmatched works.

In [20]:
import pandas as pd
import requests
from tqdm import tqdm
import re

def format_artist_name(name):
    """Format the artist name for the WikiArt API."""
    return name.lower().replace(" ", "-")

def clean_title(text):
    """Clean and normalize painting titles for comparison."""
    return re.sub(r'[^\w\s]', '', text.lower()).strip()

def fetch_paintings(artist_url):
    """Fetch all paintings for a given artist from the WikiArt API."""
    paintings = {}
    page = 1
    MAX_PAGES = 10  # Limit the number of pages to fetch
    while page <= MAX_PAGES:
        url = f"https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl={artist_url}&page={page}&json=2"
        print(f"Fetching URL: {url}")  # Debugging: Print the URL being fetched
        try:
            response = requests.get(url, timeout=10)  # Add a timeout
            if response.status_code != 200:
                print(f"Failed to fetch data. HTTP Status Code: {response.status_code}")
                print(f"Response Text: {response.text}")  # Print the response text for debugging
                break
            
            # Parse JSON response
            data = response.json()
            if not data:
                print("No more data returned from the API.")
                break
            
            # Deduplicate paintings by title
            for painting in data:
                title = painting.get('title', '').strip().lower()
                if title and title not in paintings:
                    paintings[title] = painting
            
            print(f"Page {page}: Retrieved {len(data)} paintings.")  # Debugging: Print the number of paintings retrieved
            page += 1
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            break
    
    print(f"Total unique paintings fetched for {artist_url}: {len(paintings)}")  # Debugging: Total unique paintings fetched
    return list(paintings.values())

def check_paintings_for_available_artists(file_path, available_artists):
    """Check if paintings for available artists exist in the WikiArt database."""
    # Load the dataset
    df = pd.read_csv(file_path)
    
    print("\nChecking paintings for available artists...")
    for artist in tqdm(available_artists):
        print(f"\nArtist: {artist}")
        artist_url = format_artist_name(artist)
        paintings = fetch_paintings(artist_url)
        painting_titles = [clean_title(p['title']) for p in paintings]
        
        # Filter rows for the current artist
        artist_paintings = df[df['author_name'] == artist]
        
        print(f"  Paintings for {artist}:")
        for _, row in artist_paintings.iterrows():
            painting_name = clean_title(row['painting_name'])
            if painting_name in painting_titles:
                print(f"    - {row['painting_name']}: Yes")
            else:
                print(f"    - {row['painting_name']}: No")

In [ ]:
artist_url = format_artist_name("Cennino Cennini")
paintings = fetch_paintings(artist_url)
print(f"Paintings for Cennino Cennini: {paintings}")

Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=cennino-cennini&page=1&json=2
Page 1: Retrieved 6 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=cennino-cennini&page=2&json=2
Page 2: Retrieved 6 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=cennino-cennini&page=3&json=2
Page 3: Retrieved 6 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=cennino-cennini&page=4&json=2
Page 4: Retrieved 6 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=cennino-cennini&page=5&json=2
Page 5: Retrieved 6 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=cennino-cennini&page=6&json=2
Page 6: Retrieved 6 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=cennino-cennini&page=7&json=2
Page 7: Retrieved 6 paintings.

In [22]:
import pandas as pd
from tqdm import tqdm

# List of available artists
available_artists = [
    "Cennino Cennini",
    "Giulia Lama",
    "Filippo Cifariello",
    "Yeghishe Tadevosyan",
    "Constantin Artachino",
    "Thalia Flora Karavia",
    "Matej Sternen",
    "Nikolai Kulbin",
    "Margaret Macdonald",
    "Sydney Laurence",
    "Panos Terlemezian",
    "Benjamin Brown",
    "Louis Vivin",
    "Louis Wain",
    "Rubens Santoro",
    "Mildred Anne Butler",
    "Julian Falat",
    "Eduard Von Grutzner",
    "Robert Bateman",
    "Alexandre Jacques Chantron",
    "Anton Mauve",
    "Charles Hermans",
    "Franz Richard Unterberger",
    "Henry Moore Ra",
    "Konstantin Dmitriyevich Flavitsky",
    "August Friedrich Schenck",
    "John Varley",
    "Mikhail Shibanov",
    "Christoffer Wilhelm Eckersberg",
    "Constantin Daniel Rosenthal",
    "Gustave Boulanger",
    "Manuel Rodriguez De Guzman",
    "Frederick William Burton",
    "Leonardo Alenza",
    "Charles Martin Powell",
    "Pieter Van Hanselaere",
    "Vieira Portuense",
    "Marie Guillemine Benoist",
    "Ike No Taiga",
    "Jose Luzan",
    "David Morier",
    "Yosa Buson",
    "Juriaen Van Streeck",
    "Bada Shanren",
    "Theodoor Van Thulden",
    "Johannes Moreelse",
    "Adam Van Noort",
    "Osias Beert",
    "Barbara Longhi",
    "Marietta Robusti",
    "Defendente Ferrari",
    "Tang Yin",
    "Jacques Daret",
    "Ghiberti",
    "Emperor Huizong",
    "Hildegard Of Bingen",
    "Sm Sultan",
    "Jose Manuel Capuletti",
    "Jean Francois Dubreuil",
    "Futura 2000",
    "Gregg Renfrow",
    "Hassan Massoudy",
    "Mel Ramsden",
    "Valentin Khrushch",
    "Konstantin Vasilyev",
    "Nancy Holt",
    "Hossein Zenderoudi",
    "Jean Claude Silbermann",
    "Thomas Nonn",
    "Jyoti Bhatt",
    "Constantin Blendea",
    "Afewerk Tekle",
    "Rubem Ludolf",
    "Sabin Balasa",
    "Roman Opalka",
    "Henry Villierme",
    "Grigor Khanjyan",
    "Anatoli Papian",
    "Stanley Pinker",
    "Akira Kanayama",
    "Roberto Sambonet",
    "Fernando Lanhas",
    "Jerzy Nowosielski",
    "Tia Peltz",
    "Bram Bogart",
    "Jean Benoit",
    "Albert Huie",
    "Jean Jansem",
    "Yves Laloy",
    "Maurice Boitel",
    "Vytautas Kasiulis",
    "Charles Wilbert White",
    "Hovhannes Zardaryan",
    "Ligia Macovei",
    "Paul Paun",
    "Albert Tucker",
    "Luis Dourdil",
    "Adrien Dax",
    "Graham Bell",
    "Louis Van Lint",
    "Haroutiun Galentz",
    "Paul Ackerman",
    "Petre Abrudan",
    "Octav Angheluta",
    "Raquel Forner",
    "Micaela Eleutheriade",
    "Lin Fengmian",
    "Robert Brackman",
    "Corneliu Michailescu",
    "Margareta Sterian",
    "Filippo De Pisis",
    "Carlos Orozco Romero",
    "Nutzi Acontz",
    "Friedrich Schroder Sonnenstern",
    "Alexander Bazhbeuk Melikyan",
    "Kiyokata Kaburagi",
    "Austin Osman Spare",
    "Sedrak Arakelyan",
    "Raphael Delorme",
    "Maurice Utrillo",
    "Georges Ribemont Dessaignes",
    "Max Jacob",
    "Edouard Cortes",
    "Samuel Schwarz",
    "Galileo Chini",
    "Kimon Loghi"
]

# Dictionary to store paintings for each artist
all_paintings = {}

print("\nFetching paintings for all artists...")
for artist in tqdm(available_artists):
    artist_url = format_artist_name(artist)
    paintings = fetch_paintings(artist_url)
    all_paintings[artist] = paintings

# Save the results to a JSON file
output_file = "/Users/chrisloukasntais/VsCode/EchoCanvas/data/all_paintings.json"
with open(output_file, "w") as f:
    import json
    json.dump(all_paintings, f, indent=4)

print(f"\n✅ Paintings for all artists have been saved to {output_file}")


Fetching paintings for all artists...


  0%|          | 0/126 [00:00<?, ?it/s]

Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=cennino-cennini&page=1&json=2
Page 1: Retrieved 6 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=cennino-cennini&page=2&json=2
Page 2: Retrieved 6 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=cennino-cennini&page=3&json=2
Page 3: Retrieved 6 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=cennino-cennini&page=4&json=2
Page 4: Retrieved 6 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=cennino-cennini&page=5&json=2
Page 5: Retrieved 6 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=cennino-cennini&page=6&json=2
Page 6: Retrieved 6 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=cennino-cennini&page=7&json=2
Page 7: Retrieved 6 paintings.

  1%|          | 1/126 [00:08<17:51,  8.57s/it]

Page 10: Retrieved 6 paintings.
Total unique paintings fetched for cennino-cennini: 6
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=giulia-lama&page=1&json=2
Page 1: Retrieved 12 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=giulia-lama&page=2&json=2
Page 2: Retrieved 12 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=giulia-lama&page=3&json=2
Page 3: Retrieved 12 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=giulia-lama&page=4&json=2
Page 4: Retrieved 12 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=giulia-lama&page=5&json=2
Page 5: Retrieved 12 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=giulia-lama&page=6&json=2
Page 6: Retrieved 12 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?ar

...

Page 10: Retrieved 8 paintings.
Total unique paintings fetched for samuel-schwarz: 8
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=galileo-chini&page=1&json=2
Page 1: Retrieved 12 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=galileo-chini&page=2&json=2
Page 2: Retrieved 12 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=galileo-chini&page=3&json=2
Page 3: Retrieved 12 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=galileo-chini&page=4&json=2
Page 4: Retrieved 12 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=galileo-chini&page=5&json=2
Page 5: Retrieved 12 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=galileo-chini&page=6&json=2
Page 6: Retrieved 12 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/Paintings

 99%|█████████▉| 125/126 [17:14<00:08,  8.26s/it]

Page 10: Retrieved 12 paintings.
Total unique paintings fetched for galileo-chini: 12
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=kimon-loghi&page=1&json=2
Page 1: Retrieved 42 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=kimon-loghi&page=2&json=2
Page 2: Retrieved 42 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=kimon-loghi&page=3&json=2
Page 3: Retrieved 42 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=kimon-loghi&page=4&json=2
Page 4: Retrieved 42 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=kimon-loghi&page=5&json=2
Page 5: Retrieved 42 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl=kimon-loghi&page=6&json=2
Page 6: Retrieved 42 paintings.
Fetching URL: https://www.wikiart.org/en/App/Painting/PaintingsByArtist?ar

100%|██████████| 126/126 [17:22<00:00,  8.27s/it]

Page 10: Retrieved 42 paintings.
Total unique paintings fetched for kimon-loghi: 37

✅ Paintings for all artists have been saved to /Users/chrisloukasntais/VsCode/EchoCanvas/data/all_paintings.json


## Step 12: Fetch Artist Biographies from WikiArt

### Fetch and Enrich Artist Biographies

We use the WikiArt API to collect detailed biographies for each artist. These enrichments add valuable context and narrative to our dataset, supporting deeper analysis and storytelling.

In [1]:
import requests
from tqdm import tqdm
import json

def format_artist_name(name):
    """Format the artist name for the WikiArt API."""
    return name.lower().replace(" ", "-")

def fetch_artist_biography(artist_name):
    """Fetch the full biography of an artist from the WikiArt API."""
    artist_url = format_artist_name(artist_name)
    url = f"https://www.wikiart.org/en/{artist_url}?json=2"
    response = requests.get(url)
    
    if response.status_code == 200:
        try:
            data = response.json()
            # Fetch the full biography if available
            biography = data.get("biography")
            if biography is None or biography.strip() == "":
                return "Biography not available"
            return biography
        except ValueError:
            return "Failed to parse JSON response."
    else:
        return f"Failed to fetch data. HTTP Status Code: {response.status_code}"

def fetch_biographies_for_artists(artists):
    """Fetch biographies for a list of artists."""
    artist_biographies = {}
    
    print("\nFetching biographies for artists...")
    for artist in tqdm(artists):
        biography = fetch_artist_biography(artist)
        # Normalize missing biographies
        if biography is None or biography.strip() == "":
            biography = "Biography not available"
        artist_biographies[artist] = biography
    
    return artist_biographies

# List of available artists
available_artists = [
    "Cennino Cennini",
    "Giulia Lama",
    "Filippo Cifariello",
    "Yeghishe Tadevosyan",
    "Constantin Artachino",
    "Thalia Flora Karavia",
    "Matej Sternen",
    "Nikolai Kulbin",
    "Margaret Macdonald",
    "Sydney Laurence",
    "Panos Terlemezian",
    "Benjamin Brown",
    "Louis Vivin",
    "Louis Wain",
    "Rubens Santoro",
    "Mildred Anne Butler",
    "Julian Falat",
    "Eduard Von Grutzner",
    "Robert Bateman",
    "Alexandre Jacques Chantron",
    "Anton Mauve",
    "Charles Hermans",
    "Franz Richard Unterberger",
    "Henry Moore Ra",
    "Konstantin Dmitriyevich Flavitsky",
    "August Friedrich Schenck",
    "John Varley",
    "Mikhail Shibanov",
    "Christoffer Wilhelm Eckersberg",
    "Constantin Daniel Rosenthal",
    "Gustave Boulanger",
    "Manuel Rodriguez De Guzman",
    "Frederick William Burton",
    "Leonardo Alenza",
    "Charles Martin Powell",
    "Pieter Van Hanselaere",
    "Vieira Portuense",
    "Marie Guillemine Benoist",
    "Ike No Taiga",
    "Jose Luzan",
    "David Morier",
    "Yosa Buson",
    "Juriaen Van Streeck",
    "Bada Shanren",
    "Theodoor Van Thulden",
    "Johannes Moreelse",
    "Adam Van Noort",
    "Osias Beert",
    "Barbara Longhi",
    "Marietta Robusti",
    "Defendente Ferrari",
    "Tang Yin",
    "Jacques Daret",
    "Ghiberti",
    "Emperor Huizong",
    "Hildegard Of Bingen",
    "Sm Sultan",
    "Jose Manuel Capuletti",
    "Jean Francois Dubreuil",
    "Futura 2000",
    "Gregg Renfrow",
    "Hassan Massoudy",
    "Mel Ramsden",
    "Valentin Khrushch",
    "Konstantin Vasilyev",
    "Nancy Holt",
    "Hossein Zenderoudi",
    "Jean Claude Silbermann",
    "Thomas Nonn",
    "Jyoti Bhatt",
    "Constantin Blendea",
    "Afewerk Tekle",
    "Rubem Ludolf",
    "Sabin Balasa",
    "Roman Opalka",
    "Henry Villierme",
    "Grigor Khanjyan",
    "Anatoli Papian",
    "Stanley Pinker",
    "Akira Kanayama",
    "Roberto Sambonet",
    "Fernando Lanhas",
    "Jerzy Nowosielski",
    "Tia Peltz",
    "Bram Bogart",
    "Jean Benoit",
    "Albert Huie",
    "Jean Jansem",
    "Yves Laloy",
    "Maurice Boitel",
    "Vytautas Kasiulis",
    "Charles Wilbert White",
    "Hovhannes Zardaryan",
    "Ligia Macovei",
    "Paul Paun",
    "Albert Tucker",
    "Luis Dourdil",
    "Adrien Dax",
    "Graham Bell",
    "Louis Van Lint",
    "Haroutiun Galentz",
    "Paul Ackerman",
    "Petre Abrudan",
    "Octav Angheluta",
    "Raquel Forner",
    "Micaela Eleutheriade",
    "Lin Fengmian",
    "Robert Brackman",
    "Corneliu Michailescu",
    "Margareta Sterian",
    "Filippo De Pisis",
    "Carlos Orozco Romero",
    "Nutzi Acontz",
    "Friedrich Schroder Sonnenstern",
    "Alexander Bazhbeuk Melikyan",
    "Kiyokata Kaburagi",
    "Austin Osman Spare",
    "Sedrak Arakelyan",
    "Raphael Delorme",
    "Maurice Utrillo",
    "Georges Ribemont Dessaignes",
    "Max Jacob",
    "Edouard Cortes",
    "Samuel Schwarz",
    "Galileo Chini",
    "Kimon Loghi"
]

# Fetch biographies for the available artists
artist_biographies = fetch_biographies_for_artists(available_artists)

# Save the biographies to a JSON file
output_file = "/Users/chrisloukasntais/VsCode/EchoCanvas/data/bibliographie.json"
with open(output_file, "w") as f:
    json.dump(artist_biographies, f, indent=4)

print(f"\n✅ Biographies for all artists have been saved to {output_file}")


Fetching biographies for artists...


100%|██████████| 126/126 [01:47<00:00,  1.18it/s]


✅ Biographies for all artists have been saved to /Users/chrisloukasntais/VsCode/EchoCanvas/data/bibliographie.json


## Step 13: Supplement Biographies with Wikipedia

For artists whose biographies are missing or incomplete on WikiArt, we turn to Wikipedia as an alternative source. This increases the likelihood of providing a comprehensive biography for each artist.

### Fallback: Enrich Biographies Using Wikipedia

For artists with missing or incomplete biographies on WikiArt, we turn to Wikipedia. This step increases coverage and ensures that each artist is represented with meaningful background information.

In [2]:
import requests
import wikipedia
from tqdm import tqdm
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

# Configure Wikipedia API
wikipedia.set_lang("en")

def format_artist_name(name):
    """Format artist name for WikiArt API URL."""
    return name.lower().replace(" ", "-")

def fetch_from_wikiart(artist_name):
    """Attempt to fetch artist biography from WikiArt."""
    url = f"https://www.wikiart.org/en/{format_artist_name(artist_name)}?json=2"
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            biography = data.get("biography", "").strip()
            return biography if biography else None
        return None
    except Exception:
        return None

def fetch_from_wikipedia(artist_name):
    """Fetch artist biography from Wikipedia."""
    try:
        summary = wikipedia.summary(artist_name, sentences=5, auto_suggest=True, redirect=True)
        return summary
    except wikipedia.exceptions.DisambiguationError as e:
        try:
            # Try the first non-ambiguous suggestion
            summary = wikipedia.summary(e.options[0], sentences=5)
            return summary
        except Exception:
            return None
    except Exception:
        return None

def fetch_biography(artist_name):
    """Fetch biography from WikiArt or fallback to Wikipedia."""
    bio = fetch_from_wikiart(artist_name)
    if bio:
        return bio
    else:
        return fetch_from_wikipedia(artist_name) or "Biography not available"

def fetch_all_biographies(artist_list, max_workers=10):
    """Fetch biographies in parallel with ThreadPoolExecutor."""
    biographies = {}
    print("\n🚀 Fetching biographies...")
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(fetch_biography, artist): artist for artist in artist_list}
        for future in tqdm(as_completed(futures), total=len(futures)):
            artist = futures[future]
            try:
                biographies[artist] = future.result()
            except Exception:
                biographies[artist] = "Biography not available due to error"
    return biographies

# List of artists (truncated here for brevity; use your full list)
available_artists = [  "Cennino Cennini",
    "Filippo Cifariello",
    "Constantin Artachino",
    "Thalia Flora Karavia",
    "Matej Sternen",
    "Nikolai Kulbin",
    "Margaret Macdonald",
    "Sydney Laurence",
    "Benjamin Brown",
    "Louis Vivin",
    "Louis Wain",
    "Rubens Santoro",
    "Julian Falat",
    "Eduard Von Grutzner",
    "Robert Bateman",
    "Anton Mauve",
    "Charles Hermans",
    "Franz Richard Unterberger",
    "Henry Moore Ra",
    "Konstantin Dmitriyevich Flavitsky",
    "August Friedrich Schenck",
    "John Varley",
    "Mikhail Shibanov",
    "Christoffer Wilhelm Eckersberg",
    "Constantin Daniel Rosenthal",
    "Gustave Boulanger",
    "Manuel Rodriguez De Guzman",
    "Frederick William Burton",
    "Leonardo Alenza",
    "Charles Martin Powell",
    "Pieter Van Hanselaere",
    "Vieira Portuense",
    "Marie Guillemine Benoist",
    "Ike No Taiga",
    "Yosa Buson",
    "Juriaen Van Streeck",
    "Theodoor Van Thulden",
    "Johannes Moreelse",
    "Adam Van Noort",
    "Osias Beert",
    "Barbara Longhi",
    "Marietta Robusti",
    "Defendente Ferrari",
    "Tang Yin",
    "Jacques Daret",
    "Sm Sultan",
    "Jose Manuel Capuletti",
    "Jean Francois Dubreuil",
    "Futura 2000",
    "Gregg Renfrow",
    "Mel Ramsden",
    "Valentin Khrushch",
    "Konstantin Vasilyev",
    "Nancy Holt",
    "Hossein Zenderoudi",
    "Jean Claude Silbermann",
    "Jyoti Bhatt",
    "Constantin Blendea",
    "Afewerk Tekle",
    "Rubem Ludolf",
    "Sabin Balasa",
    "Roman Opalka",
    "Henry Villierme",
    "Stanley Pinker",
    "Akira Kanayama",
    "Roberto Sambonet",
    "Fernando Lanhas",
    "Jerzy Nowosielski",
    "Tia Peltz",
    "Bram Bogart",
    "Jean Benoit",
    "Albert Huie",
    "Yves Laloy",
    "Maurice Boitel",
    "Charles Wilbert White",
    "Ligia Macovei",
    "Paul Paun",
    "Luis Dourdil",
    "Adrien Dax",
    "Graham Bell",
    "Louis Van Lint",
    "Paul Ackerman",
    "Petre Abrudan",
    "Octav Angheluta",
    "Filippo De Pisis",
    "Carlos Orozco Romero",
    "Nutzi Acontz",
    "Friedrich Schroder Sonnenstern",
    "Kiyokata Kaburagi",
    "Austin Osman Spare",
    "Raphael Delorme",
    "Maurice Utrillo",
    "Georges Ribemont Dessaignes",
    "Edouard Cortes",
    "Samuel Schwarz",
    "Galileo Chini",
    "Kimon Loghi"
]

# Fetch and save biographies
biographies = fetch_all_biographies(available_artists, max_workers=20)

output_file = "/Users/chrisloukasntais/VsCode/EchoCanvas/data/bibliographie.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(biographies, f, indent=4, ensure_ascii=False)

print(f"\n✅ Biographies saved successfully to {output_file}")



🚀 Fetching biographies...


  8%|▊         | 8/97 [00:03<00:18,  4.92it/s]/Users/chrisloukasntais/.pyenv/versions/3.11.9/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/chrisloukasntais/.pyenv/versions/3.11.9/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
100%|██████████| 97/97 [00:17<00:00,  5.63it/s]


✅ Biographies saved successfully to /Users/chrisloukasntais/VsCode/EchoCanvas/data/bibliographie.json


## Step 14: Fill Remaining Biographies Using Wikidata

If both WikiArt and Wikipedia lack a biography for an artist, we use the Wikidata API as a final fallback. This ensures that as many artists as possible have at least a brief description.

### Final Step: Fill Remaining Biographies from Other Sources

If some artists still lack biographies after WikiArt and Wikipedia, we use Wikidata and other open databases to fill the gaps. This ensures our dataset is as complete and informative as possible.

In [6]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import json

# Painters missing bios
painters = [
    "Franz Richard Unterberger", "Margaret Macdonald", "Louis Wain", "Sydney Laurence", "Benjamin Brown",
    "Rubem Ludolf", "Roberto Sambonet", "Akira Kanayama", "Fernando Lanhas", "Yves Laloy", "Luis Dourdil",
    "Adrien Dax", "Petre Abrudan", "Octav Angheluta", "Futura 2000", "Gregg Renfrow", "Nancy Holt",
    "Jean Claude Silbermann", "Constantin Blendea", "Jean Francois Dubreuil", "Sm Sultan", "Jacques Daret"
]

# Wikidata biography fetcher
def get_wikidata_bio(name):
    try:
        search_url = "https://www.wikidata.org/w/api.php"
        search_params = {
            "action": "wbsearchentities",
            "search": name,
            "language": "en",
            "format": "json"
        }
        search_resp = requests.get(search_url, params=search_params, timeout=10)
        search_data = search_resp.json()

        if not search_data.get("search"):
            return name, "Biography not available"

        entity_id = search_data["search"][0]["id"]
        data_url = f"https://www.wikidata.org/wiki/Special:EntityData/{entity_id}.json"
        data_resp = requests.get(data_url, timeout=10)
        entity_data = data_resp.json()

        description = entity_data["entities"][entity_id].get("descriptions", {}).get("en", {}).get("value")
        return name, description if description else "Biography not available"

    except Exception as e:
        return name, "Biography not available"

# Parallel fetching
def fetch_all_wikidata(painters, max_workers=8):
    results = {}
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_name = {executor.submit(get_wikidata_bio, name): name for name in painters}
        for future in as_completed(future_to_name):
            name, bio = future.result()
            results[name] = bio
    return results

# File path
json_file_path = "/Users/chrisloukasntais/VsCode/EchoCanvas/data/bibliographie.json"

# Load the existing biography dictionary
with open(json_file_path, "r", encoding="utf-8") as f:
    bio_data = json.load(f)

# Determine which authors need updating
authors_to_update = [name for name in painters if bio_data.get(name) == "Biography not available"]

print(f"🔎 Fetching biographies for {len(authors_to_update)} painters from Wikidata...")

# Run the fetch
start = time.time()
updated_bios = fetch_all_wikidata(authors_to_update)
print(f"✅ Fetch completed in {time.time() - start:.2f} seconds")

# Update the JSON data
for name, biography in updated_bios.items():
    bio_data[name] = biography

# Save the updated dictionary
with open(json_file_path, "w", encoding="utf-8") as f:
    json.dump(bio_data, f, indent=4, ensure_ascii=False)

print(f"\n📝 Biographies updated and saved to: {json_file_path}")


🔎 Fetching biographies for 20 painters from Wikidata...
✅ Fetch completed in 3.39 seconds

📝 Biographies updated and saved to: /Users/chrisloukasntais/VsCode/EchoCanvas/data/bibliographie.json


## Step 15: Identify Artists Still Missing Biographies

### Summary: Remaining Missing Biographies

After all enrichment steps, only a few artists may still lack biographies. We document these cases for transparency and potential future updates.

In [1]:
import requests
import wikipedia
import json

# Set Wikipedia language to English
wikipedia.set_lang("en")

def fetch_wikipedia_bio(name):
    try:
        return wikipedia.summary(name, sentences=5, auto_suggest=True, redirect=True)
    except wikipedia.exceptions.DisambiguationError as e:
        try:
            return wikipedia.summary(e.options[0], sentences=5)
        except Exception:
            return None
    except Exception:
        return None

def fetch_wikidata_bio(name):
    try:
        search_url = "https://www.wikidata.org/w/api.php"
        search_params = {
            "action": "wbsearchentities",
            "search": name,
            "language": "en",
            "format": "json"
        }
        search_resp = requests.get(search_url, params=search_params, timeout=10)
        search_data = search_resp.json()
        if not search_data.get("search"):
            return None
        entity_id = search_data["search"][0]["id"]
        data_url = f"https://www.wikidata.org/wiki/Special:EntityData/{entity_id}.json"
        data_resp = requests.get(data_url, timeout=10)
        entity_data = data_resp.json()
        description = entity_data["entities"][entity_id].get("descriptions", {}).get("en", {}).get("value")
        return description
    except Exception:
        return None

# Load the JSON file
json_file_path = "/Users/chrisloukasntais/VsCode/EchoCanvas/data/bibliographie.json"
with open(json_file_path, "r", encoding="utf-8") as f:
    bio_data = json.load(f)

# Painters to update
painters = ["Kimon Loghi", "Tang Yin", "John Varley"]

for name in painters:
    if bio_data.get(name, "Biography not available") == "Biography not available":
        # Try Wikipedia first
        bio = fetch_wikipedia_bio(name)
        if not bio:
            # Try Wikidata if Wikipedia fails
            bio = fetch_wikidata_bio(name)
        if not bio:
            bio = "Biography not available"
        bio_data[name] = bio

# Save the updated JSON
with open(json_file_path, "w", encoding="utf-8") as f:
    json.dump(bio_data, f, indent=4, ensure_ascii=False)

print("✅ Biographies for Kimon Loghi, Tang Yin, and John Varley have been updated in bibliographie.json")

✅ Biographies for Kimon Loghi, Tang Yin, and John Varley have been updated in bibliographie.json


## Step 16: Enrich Short Biographies

Some artist biographies may be very brief (e.g., less than 100 characters). We attempt to enrich these by fetching more detailed summaries from Wikipedia, ensuring every artist profile is informative.

In [3]:
import wikipedia
import json

json_file_path = "/Users/chrisloukasntais/VsCode/EchoCanvas/data/bibliographie.json"
with open(json_file_path, "r", encoding="utf-8") as f:
    bio_data = json.load(f)

wikipedia.set_lang("en")
SHORT_BIO_LENGTH = 100

def try_wikipedia(name):
    # Try exact match first
    try:
        return wikipedia.summary(name, sentences=5, auto_suggest=False, redirect=True)
    except wikipedia.exceptions.DisambiguationError as e:
        # Try the first few options if available
        for option in e.options[:3]:
            try:
                return wikipedia.summary(option, sentences=5, auto_suggest=False)
            except Exception:
                continue
        return None
    except Exception:
        # Try with auto_suggest as fallback
        try:
            return wikipedia.summary(name, sentences=5, auto_suggest=True, redirect=True)
        except Exception:
            return None

for name, bio in bio_data.items():
    if bio and len(bio) < SHORT_BIO_LENGTH:
        summary = try_wikipedia(name)
        if summary and len(summary) > len(bio):
            bio_data[name] = summary
            print(f"Updated biography for {name}")
        else:
            print(f"Could not fetch Wikipedia summary for {name}")

with open(json_file_path, "w", encoding="utf-8") as f:
    json.dump(bio_data, f, indent=4, ensure_ascii=False)

Could not fetch Wikipedia summary for Franz Richard Unterberger
Updated biography for Margaret Macdonald
Updated biography for Louis Wain
Updated biography for Sydney Laurence
Updated biography for John Varley
Updated biography for Benjamin Brown
Updated biography for Tang Yin
Updated biography for Futura 2000
Could not fetch Wikipedia summary for Sm Sultan
Could not fetch Wikipedia summary for Gregg Renfrow
Updated biography for Nancy Holt
Could not fetch Wikipedia summary for Jean Claude Silbermann
Could not fetch Wikipedia summary for Constantin Blendea
Could not fetch Wikipedia summary for Rubem Ludolf
Could not fetch Wikipedia summary for Roberto Sambonet
Updated biography for Akira Kanayama
Updated biography for Fernando Lanhas
Could not fetch Wikipedia summary for Yves Laloy
Could not fetch Wikipedia summary for Luis Dourdil
Could not fetch Wikipedia summary for Adrien Dax
Could not fetch Wikipedia summary for Petre Abrudan
Could not fetch Wikipedia summary for Octav Angheluta
C

## Step 17: Manually Update Remaining Short Biographies

### Manually Add Biographies for Remaining Artists

For any artists still lacking sufficient biographical information, we manually add or edit their entries to ensure the dataset is comprehensive and ready for public use.

In [4]:
import json

json_file_path = "/Users/chrisloukasntais/VsCode/EchoCanvas/data/bibliographie.json"
with open(json_file_path, "r", encoding="utf-8") as f:
    bio_data = json.load(f)

SHORT_BIO_LENGTH = 100

short_bio_painters = [name for name, bio in bio_data.items() if bio and len(bio) < SHORT_BIO_LENGTH]

print("Painters with biographies less than 100 characters:")
for name in short_bio_painters:
    print(f"- {name}: {bio_data[name]}")

Painters with biographies less than 100 characters:
- Franz Richard Unterberger: painter from Austria
- John Varley: American science fiction author
- Benjamin Brown: American physician and politician, Massachusetts (1756-1831)
- Sm Sultan: Bangladeshi artist (1923-1994)
- Gregg Renfrow: US American painter and sculptor
- Jean Claude Silbermann: French writer and painter
- Constantin Blendea: Romanian painter (1929-2012)
- Rubem Ludolf: Brazilian painter
- Roberto Sambonet: Italian designer, architect and painter (1924-1995)
- Yves Laloy: French architect and painter (1920-1999)
- Luis Dourdil: Portuguese painter (1914-1989)
- Adrien Dax: French painter (1913-1979)
- Petre Abrudan: Romanian painter (1907-1979)
- Octav Angheluta: painter (1904-1979)
- Kimon Loghi: Romanian painter (1873-1952)


### Review for Short Biographies

After enriching and cleaning the biographies, it's important to check if any artists still have very brief biographies (less than 100 characters). This helps us identify entries that may need further manual attention or additional research to ensure every artist is well represented in the dataset.

In [3]:
import json

json_file_path = "/Users/chrisloukasntais/VsCode/EchoCanvas/data/bibliographie.json"
with open(json_file_path, "r", encoding="utf-8") as f:
    bio_data = json.load(f)

SHORT_BIO_LENGTH = 100

short_bio_painters = [name for name, bio in bio_data.items() if bio and len(bio) < SHORT_BIO_LENGTH]

print("Painters with biographies less than 100 characters:")
for name in short_bio_painters:
    print(f"- {name}: {bio_data[name]}")

Painters with biographies less than 100 characters:


### Beautify and Clean Biography Text

To make the artist biographies more readable and visually appealing, we remove unwanted formatting such as extra slashes, line breaks (\n), and Wikipedia section headers (like '== Biography =='). This step ensures that each biography is presented as a clean, well-structured paragraph, enhancing the quality and professionalism of the dataset.

In [1]:
import json
import re

json_file_path = "/Users/chrisloukasntais/VsCode/EchoCanvas/data/bibliographie.json"

with open(json_file_path, "r", encoding="utf-8") as f:
    bio_data = json.load(f)

def clean_bio(text):
    # Remove Wikipedia section headers like == Biography ==, == Life ==, etc.
    text = re.sub(r"==.*?==", "", text)
    # Replace all \n with a space
    text = text.replace("\\n", " ").replace("\n", " ")
    # Remove multiple spaces
    text = re.sub(r"\s+", " ", text)
    # Remove leading/trailing whitespace
    return text.strip()

for name in bio_data:
    bio_data[name] = clean_bio(bio_data[name])

# Save cleaned biographies
output_file = "/Users/chrisloukasntais/VsCode/EchoCanvas/data/bibliographie_clean.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(bio_data, f, indent=4, ensure_ascii=False)

print(f"Cleaned biographies saved to {output_file}")

Cleaned biographies saved to /Users/chrisloukasntais/VsCode/EchoCanvas/data/bibliographie_clean.json
